# LangChain Expression Language (LCEL)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole, chat_completion

_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GIGACHAT_API_KEY')

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_gigachat import GigaChat
from langchain.schema.output_parser import StrOutputParser

## Простейшая цепочка (Simple Chain)

In [19]:
prompt = ChatPromptTemplate.from_template(
    "Расскажи короткую шутку о {topic}"
)
model = GigaChat(credentials=api_key, verify_ssl_certs=False)
output_parser = StrOutputParser()

In [20]:
chain = prompt | model | output_parser

In [21]:
chain.invoke({"topic": "медведи"})

'Почему медведь не играет в шахматы?\n \n Потому что он предпочитает охотиться на шахматную доску!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [25]:
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

C:\Users\Maksim\AppData\Local\Temp\ipykernel_10072\554499278.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

C:\Users\Maksim\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Maksim\.cache\huggingface\hub\models--intfloat--multilingual-e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [29]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Харрисон работал в хлеву", "медведи любят есть мед"],
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()

C:\Users\Maksim\AppData\Roaming\Python\Python311\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [30]:
retriever.get_relevant_documents("Где работал Харрисон?")

C:\Users\Maksim\AppData\Local\Temp\ipykernel_10072\2773580541.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Где работал Харрисон?")


[Document(metadata={}, page_content='Харрисон работал в хлеву'),
 Document(metadata={}, page_content='медведи любят есть мед')]

In [31]:
retriever.get_relevant_documents("Что медведям нравится есть")

[Document(metadata={}, page_content='медведи любят есть мед'),
 Document(metadata={}, page_content='Харрисон работал в хлеву')]

In [32]:
template = """Ответь на вопрос ориентируясь только на представленный далее контекст:
{context}

Вопрос: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain.schema.runnable import RunnableMap

In [34]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [40]:
chain.invoke({"question": "Где работал Харрисон?"})

'Харрисон работал в хлеву.'

In [43]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [44]:
chain.invoke({"question": "Где работал Харрисон?"})

'Харрисон работал в хлеву.'

In [36]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [37]:
inputs.invoke({"question": "Где работал Харрисон?"})

{'context': [Document(metadata={}, page_content='Харрисон работал в хлеву'),
  Document(metadata={}, page_content='медведи любят есть мед')],
 'question': 'Где работал Харрисон?'}

## Bind

and OpenAI Functions

In [45]:
functions = [
    {
      "name": "weather_search",
      "description": "Узнай погоду по коду аэропорта",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "Код аэропорта для узнавания там погоды"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [47]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = GigaChat(credentials=api_key, verify_ssl_certs=False, temperature=0)\
        .bind(functions=functions)

In [48]:
runnable = prompt | model

In [53]:
runnable.invoke({"input": "Какая погода в Санкт-Петербурге"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': '4c5f00e1-a1b7-4c32-ae60-0df06f40105c'}, response_metadata={'token_usage': {'prompt_tokens': 83, 'completion_tokens': 29, 'total_tokens': 112}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-7ee2a77a-ca60-4dfd-9379-14dc5958568c-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'LED'}, 'id': '41fe16e8-7157-48fe-9fb3-6973e79874a9', 'type': 'tool_call'}])

In [54]:
functions = [
    {
      "name": "weather_search",
      "description": "Узнай погоду по коду аэропорта",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "Код аэропорта для узнавания там погоды"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Поиск новостей о последних спортивных событиях",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "Название спортивной команды для поиска"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [55]:
model = model.bind(functions=functions)

In [56]:
runnable = prompt | model

In [59]:
runnable.invoke({"input": "Что делал футбольная команда из СПБ вчера?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': {'team_name': 'Зенит'}}, 'functions_state_id': '80fdf0cd-d2a6-417e-ade9-f703e1c23a4b'}, response_metadata={'token_usage': {'prompt_tokens': 146, 'completion_tokens': 29, 'total_tokens': 175}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-90f7a08f-8000-4f4b-9228-3bad0f811c13-0', tool_calls=[{'name': 'sports_search', 'args': {'team_name': 'Зенит'}, 'id': '8cbdc788-39a1-494e-9124-d23f0bb6c8ee', 'type': 'tool_call'}])

In [60]:
runnable.invoke({"input": "Что делал Зенит вчера?"})

AIMessage(content='К сожалению, я не могу предоставить информацию о действиях конкретного футбольного клуба "Зенит" в реальном времени, так как у меня нет доступа к интернету и актуальным данным. Рекомендую проверить новостные сайты или официальные источники футбольного клуба "Зенит", чтобы узнать последние новости и результаты матчей.', additional_kwargs={'functions_state_id': '1e962837-8277-4802-816b-622555a96b4d'}, response_metadata={'token_usage': {'prompt_tokens': 142, 'completion_tokens': 72, 'total_tokens': 214}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-2c5c64d5-be89-42ce-94d4-003d5e050b51-0')

In [58]:
runnable.invoke({"input": "Какая погода в Санкт-Петербурге"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': 'b5aa160d-1470-48dd-b0a7-e7df97e64fd8'}, response_metadata={'token_usage': {'prompt_tokens': 144, 'completion_tokens': 29, 'total_tokens': 173}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-dfa59078-1446-4fc3-b7a4-a84b5aa7650c-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'LED'}, 'id': 'cf2c9db9-7a81-40b4-b2bf-917b4654b128', 'type': 'tool_call'}])

## Fallbacks

In [64]:
import json

model = GigaChat(credentials=api_key, verify_ssl_certs=False)

In [77]:
simple_chain = model | json.loads

In [102]:
challenge = "Напишите три стихотворения в виде файла json, где каждое стихотворение представляет собой двоичный файл json с названием, \
             автором и первой строкой. Весь JSON должен находится тегами <JSON>.\
            \
            Пример:\
            <JSON>\
                {\
                    \"poems\": [\
                        {\
                        \"title\": \"Одинокая звезда\",\
                        \"author\": \"Александр Сергеевич Пушкин\",\
                        \"first_line\": \"Светит одна звезда в ночной тиши...\"\
                        },\
                        {\
                        \"title\": \"Ветер свободы\",\
                        \"author\": \"Анна Ахматова\",\
                        \"first_line\": \"Ветер свободы поет над полями...\"\
                        },\
                        {\
                        \"title\": \"Шёпот листьев\",\
                        \"author\": \"Федор Иванович Тютчев\",\
                        \"first_line\": \"Шёпот, робкое дыханье...\"\
                        }\
                    ]\
                }\
            <\JSON>\
            \
            Твой ответ:\
            <JSON>\
"

In [103]:
response = model.invoke(challenge)

In [104]:
response

AIMessage(content='К сожалению, я не могу создать JSON-файлы без использования дополнительных навыков. Если у вас есть конкретные требования к содержимому этих файлов (например, сами стихи), пожалуйста, предоставьте их, и я помогу вам сгенерировать нужный JSON.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 242, 'completion_tokens': 61, 'total_tokens': 303}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-dcc35b3d-ef9c-44ff-91e9-0d1e2e57e6fb-0')

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [69]:
simple_chain.invoke(challenge)

TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

In [81]:
chain = model | StrOutputParser() | json.loads

In [82]:
chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Вообще, по хорошему, модель если бы выдала JSON (а я не смог её заставить это сделать, как ни пытался. Точнее, она либо выдавала его в не лучшем форматировании, с лишним текстом, либо вообще отказывалась генерировать его), дальнейшие действия цепочки распрарсили бы этот JSON

In [72]:
final_chain = simple_chain.with_fallbacks([chain])

In [73]:
final_chain.invoke(challenge)

TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

## Interface

In [107]:
prompt = ChatPromptTemplate.from_template(
    "Расскажи мне короткую шутку о {topic}"
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [108]:
chain.invoke({"topic": "медведях"})

"Почему медведь сидит в углу?\n because he doesn't want to be the bear of the party!"

In [109]:
chain.batch([{"topic": "медведях"}, {"topic": "лягушках"}])

['Почему медведь сидит в углу?\n\nПотому что его баскетбольная команда проигрывает! 😄',
 'Почему лягушка не смогла пойти на вечеринку?\n\nПотому что она уже была кваканой!']

In [110]:
for t in chain.stream({"topic": "медведях"}):
    print(t)

Почему медведь сидит в углу?

 Потому что он не хочет быть в центре внимания!



In [111]:
response = await chain.ainvoke({"topic": "медведях"})
response

'Почему медведь сидит в углу?\n Потому что он не хочет быть в центре внимания!'